### Fee Calculator API Fee Retrieve
Retrieves some standard fee results from LAA Fee Calculator    
Store results in DataFrame and display with some highlighting     

**Results are displayed near the end**

*Launching local fee calc*    
`DEBUG="True" venv/bin/python manage.py runserver 8000`

### Setup some stuff

In [49]:
import time
import requests
import pandas as pd

In [50]:
def get_fee(scheme = 2,
            fee_type="LIT_FEE",
            scenario="1",
            offence_class="A",
            days="1",
            ppe="0",
            app_url = "http://localhost:8000/"):
    """Get free using LAA Fee Calculator API"""
    
    # create end-part of URL
    end = ("api/v1/fee-schemes/{}/calculate/"
           "?fee_type_code={}&scenario={}&offence_class={}&day={}&ppe={}"
           ).format(scheme, fee_type, scenario, offence_class, days, ppe)
 
    response = requests.get(app_url + end)
    if response.status_code == 200:
        # .json method returns a dictionary - use to extract just the amount
        return response.json().get("amount")
    else:
        return "error: {} - {}".format(response.status_code, response.text)

In [51]:
def highlight_conditions(val):
    """Pandas dataframe conditional highlighting function
    Used with df.style.applymap method.
    """
    colour = "black"
    if "error" in str(val):
        colour = "red"
    return 'color: %s' % colour

### Interact with API to extract a range of responses and store in DataFrame

Lit Fee, Scenario 1, 1-day duration    
Varying Offence Codes and PPE values (including some invalid values).

In [54]:
#DataFrame to hold results
df = pd.DataFrame(columns=["Offence Code", "PPE", "Result"])

app_url = "http://localhost:8000/"

start_time = time.strftime("%d/%m/%Y %H:%M:%S")
for oc in "ABCDEFGHIJKL":
    for ppe in [-81, -80, -1, 0, 1, 80, 81, "z"]:
        result = get_fee(app_url=app_url, offence_class=oc, ppe=ppe)
        row_data = {"Offence Code":oc, "PPE":ppe, "Result":result}
        df = df.append(row_data, ignore_index=True)

### Display Results

In [55]:
# Highlight errors in red
highlighted = df.style.applymap(highlight_conditions)
# Turn of row display limit
pd.options.display.max_rows = None
# Display results
display(start_time)
display(highlighted)

'22/11/2018 10:19:02'

,Offence Code,PPE,Result
0,A,-81,340.2
1,A,-80,340.2
2,A,-1,340.2
3,A,0,340.2
4,A,1,340.2
5,A,80,340.2
6,A,81,343
7,A,z,"error: 400 - [""`ppe` must be a number""]"
8,B,-81,278.06
9,B,-80,278.06
